https://www.kaggle.com/uciml/student-alcohol-consumption?select=student-mat.csv

https://www.kaggle.com/davidjfisher/illinois-doc-labeled-faces-dataset

In [17]:
!pip install pandas
!pip install matplotlib
!pip install scipy
!pip install os
!pip install numpy

  Could not find a version that satisfies the requirement os (from versions: )
No matching distribution found for os


In [18]:
import pandas as pd
import matplotlib
import scipy
import os
import numpy as np

In [19]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

Scraped from the Illinois DOC.

https://www.idoc.state.il.us/subsections/search/inms_print.asp?idoc=
https://www.idoc.state.il.us/subsections/search/pub_showfront.asp?idoc=
https://www.idoc.state.il.us/subsections/search/pub_showside.asp?idoc=

paste <(cat ids.txt | sed 's/^/http:\/\/www.idoc.state.il.us\/subsections\/search\/pub_showside.asp\?idoc\=/g') <(cat ids.txt| sed 's/^/  out=/g' | sed 's/$/.jpg/g') -d '\n' > showside.txt 
paste <(cat ids.txt | sed 's/^/http:\/\/www.idoc.state.il.us\/subsections\/search\/pub_showfront.asp\?idoc\=/g') <(cat ids.txt| sed 's/^/  out=/g' | sed 's/$/.jpg/g') -d '\n' > showfront.txt      
paste <(cat ids.txt | sed 's/^/http:\/\/www.idoc.state.il.us\/subsections\/search\/inms_print.asp\?idoc\=/g') <(cat ids.txt| sed 's/^/  out=/g' | sed 's/$/.html/g') -d '\n' > inmates_print.txt 

aria2c -i ../inmates_print.txt -j4 -x4 -l ../log-$(pwd|rev|cut -d/ -f 1|rev)-$(date +%s).txt

Then use htmltocsv.py to get the csv. Note that the script is very poorly written and may have errors. It also doesn't do anything with the warrant-related info, although there are some commented-out lines which may be relevant.
Also note that it assumes all the HTML files are located in the inmates directory., and overwrites any csv files in csv if there are any.

front.7z contains mugshots from the front
side.7z contains mugshots from the side
inmates.7z contains all the html files
csv contains the html files converted to CSV

The reason for packaging the images is that many torrent clients would otherwise crash if attempting to load the torrent.

All CSV files contain headers describing the nature of the columns. For person.csv, the id is unique. For marks.csv and sentencing.csv, it is not.
Note that the CSV files use semicolons as delimiters and also end with a trailing semicolon. If this is unsuitable, edit the arr2csvR function in htmltocsv.py.

There are 68149 inmates in total, although some (a few hundred) are marked as "Unknown"/"N/A"/"" in one or more fields.

The "height" column has been processed to contain the height in inches, rather than the height in feet and inches expressed as "X ft YY in."
Some inmates were marked "Not Available", this has been replaced with "N/A".
Likewise, the "weight" column has been altered "XXX lbs." -> "XXX". Again, some are marked "N/A".

The "date of birth" column has some inmates marked as "Not Available" and others as "". There doesn't appear to be any pattern. It may be related to the institution they are kept in. Otherwise, the format is MM/DD/YYYY.

The "weight" column is often rounded to the nearest 5 lbs.

Statistics for hair:
  43305 Black
  17371 Brown
   2887 Blonde or Strawberry
   2539 Gray or Partially Gray
    740 Red or Auburn
    624 Bald
    396 Not Available
    209 Salt and Pepper
     70 White
      7 Sandy
      1 Unknown

Statistics for sex:
  63409 Male
   4740 Female

Statistics for race:
  37991 Black
  20992 White
   8637 Hispanic
    235 Asian
    104 Amer Indian
     94 Unknown
     92 Bi-Racial
      4 

Statistics for eyes:
  51714 Brown
   7808 Blue
   4259 Hazel
   2469 Green
   1382 Black
    420 Not Available
     87 Gray

In [20]:
person = pd.read_excel('person.xlsx')
marks = pd.read_excel('marks.xlsx')
sentencing = pd.read_excel('sentencing.xlsx')

In [21]:
os.listdir()

['.ipynb_checkpoints',
 '436801_829570_bundle_archive.zip',
 'datasets',
 'datasets_251_561_student-mat.csv',
 'Data_cleaning.ipynb',
 'desktop.ini',
 'front',
 'htmltocsv.py',
 'illinois_doc_dataset.torrent',
 'images',
 'inmates',
 'M1',
 'M2',
 'marks.csv',
 'marks.xlsx',
 'marks_2',
 'merged',
 'merged_person_sentencing.csv',
 'notebook_M1D4.ipynb',
 'person.csv',
 'person.xlsx',
 'persons_2',
 'Python for ML',
 'readme',
 'sentencing.csv',
 'sentencing.xlsx',
 'sentencing_2',
 'sentencing_2.xlsx',
 'side',
 'Untitled.ipynb',
 'Untitled1.ipynb']

In [22]:
person.head()

,id,name,date_of_birth,weight,hair,sex,height,race,eyes,admission_date,...,last_paroled_date,projected_discharge_date,parole_date,electronic_detention_date,discharge_date,parent_institution,offender_status,location,sex_offender_registry_required,alias
0,A00147,"MCCUTCHEON, JOHN",06/14/1949,185.0,Brown,Male,67.0,White,Blue,02/16/1983,...,NaN,2036-06-10 00:00:00,NaN,NaN,NaN,DIXON CORRECTIONAL CENTER,IN CUSTODY,DIXON,1.0,NaN
1,A00220,"WALKER, ISIAH",03/30/1957,155.0,Black,Male,73.0,Black,Brown,05/19/2016,...,NaN,NaN,NaN,NaN,NaN,STATEVILLE CORRECTIONAL CENTER,NON-IDOC CUSTODY,ILL/OTH STATE/FED CONCURR,NaN,NaN
2,A00360,"BELL, HOWARD",12/18/1946,167.0,Gray or Partially Gray,Male,69.0,White,Green,02/26/1988,...,NaN,TO BE DETERMINED,2017-02-10 00:00:00,NaN,NaN,PINCKNEYVILLE CORRECTIONAL CENTER,PAROLE,PAROLE DISTRICT 1,NaN,HOWARD R BELL | DONALD BROADSTONE | RONALD B...
3,A00367,"GARVIN, RAYMOND",1954-12-01 00:00:00,245.0,Black,Male,72.0,Black,Brown,2017-09-11 00:00:00,...,NaN,11/20/2020,NaN,NaN,NaN,WESTERN ILLINOIS CORRECTIONAL CENTER,IN CUSTODY,WESTERN ILLINOIS,NaN,NaN
4,A01054,"TIPTON, DARNELL",03/25/1954,166.0,Salt and Pepper,Male,67.0,Black,Brown,12/23/1988,...,NaN,08/14/2068,NaN,NaN,NaN,MENARD CORRECTIONAL CENTER,IN CUSTODY,MENARD,1.0,NaN


In [23]:
sentencing.head()

,id,mittimus,class,count,offense,custody_date,sentence,county,sentence_discharged
0,A00147,82CF32,1,2,AGGR KIDNAPPING-NO RANSOM,11/26/1982,30 Years 0 Months 0 Days,JASPER,NO
1,A00147,82CF32,X,1,ATTEMPT MURDER/INTENT TO KILL/INJURE,11/26/1982,50 Years 0 Months 0 Days,JASPER,NO
2,A00147,82CF32,X,1,RAPE,11/26/1982,50 Years 0 Months 0 Days,JASPER,NO
3,A00147,82CF32,2,1,BURGLARY,11/26/1982,14 Years 0 Months 0 Days,JASPER,NO
4,A00220,7916505,3,1,ATTEMPT BURGLARY,00/00/0000,2 Years 0 Months 0 Days,COOK,YES


In [24]:
marks.head()

,id,mark,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,A00147,"SCAR, ARM, RIGHT - 2\""",NaN,NaN,NaN,NaN
1,A00147,"SCAR, WRIST, RIGHT - 1\""",NaN,NaN,NaN,NaN
2,A00147,"TATTOO, ARM, LEFT - 1111",NaN,NaN,NaN,NaN
3,A00220,"SCAR, ARM, LEFT - SCAR ARM, LEFT",NaN,NaN,NaN,NaN
4,A00220,"SCAR, CHEST - SCAR CHEST",NaN,NaN,NaN,NaN


In [25]:
marks_2 = marks#[0:49]
marks_2.head()

,id,mark,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,A00147,"SCAR, ARM, RIGHT - 2\""",NaN,NaN,NaN,NaN
1,A00147,"SCAR, WRIST, RIGHT - 1\""",NaN,NaN,NaN,NaN
2,A00147,"TATTOO, ARM, LEFT - 1111",NaN,NaN,NaN,NaN
3,A00220,"SCAR, ARM, LEFT - SCAR ARM, LEFT",NaN,NaN,NaN,NaN
4,A00220,"SCAR, CHEST - SCAR CHEST",NaN,NaN,NaN,NaN


## Marks

In [26]:
print('There are ' + 
      str(len(marks_2['mark'].unique())) + 
      ' unique strings in a total of ' + 
      str(len(marks_2)) + 
      ' strings in the column mark, ' +
      'so I need to organise these'
     )

There are 141907 unique strings in a total of 192747 strings in the column mark, so I need to organise these


Sorting for visible scars, tattoos and medical conditions

In [27]:
marks_2['scar_visible'] = ""
marks_2['tat_visible'] = ""
marks_2['mc_visible'] = ""
marks_2['any_visible'] = ""

substring_list_scar = ['SCAR, ARM', 'SCAR ARM', 
                 'SCAR, FOREHEAD', 'SCAR FOREHEAD',
                 'SCAR, HAND', 'SCAR HAND',
                 'SCAR, NECK', 'SCAR NECK',
                 'SCAR, WRIST', 'SCAR WRIST',
                 'SCAR, CHIN', 'SCAR CHIN',
                 'TATTOO, EYEBROWS', 'TATTOO EYEBROWS',
                 'TATTOO, EYEBROW', 'TATTOO EYEBROW',
                 'TATTOO, BROW', 'TATTOO BROW',
                 'MISSING LEG', 
                 'MISSING ARM',
                 'MISSING HAND', 
                 'MISSING FOOT',
                 'MISSING FINGER'
                 'MISSING EYE',
                 'MISSING TOOTH',
                 'MISSING TEETH',
                 'MISSING EAR'
                 ]

substring_list_tat = ['TATTOO, ARM', 'TATTOO ARM', 
                 'TATTOO, FOREHEAD', 'TATTOO FOREHEAD',
                 'TATTOO, HAND', 'TATTOO HAND',
                 'TATTOO, NECK', 'TATTOO NECK',
                 'TATTOO, WRIST', 'TATTOO WRIST',
                 'TATTOO, CHIN', 'TATTOO CHIN',
                 'TATTOO, EAR', 'TATTOO EAR',
                 'TATTOO, BROW', 'TATTOO BROW',
                 'TATTOO, EYEBROW', 'TATTOO EYEBROW',
                 'TATTOO, EYEBROWS', 'TATTOO EYEBROWS',
                 'TATTOO, CHEEK', 'TATTOO CHEEK',
                 ]

substring_list_mc = ['ACNE'   
                 ]

for row in range(len(marks_2)):
    if any(substring in marks_2['mark'][row] for substring in substring_list_scar) == True:
        marks_2['scar_visible'][row] = 1
    else:
        marks_2['scar_visible'][row] = 0



for row in range(len(marks_2)):
    if any(substring in marks_2['mark'][row] for substring in substring_list_tat) == True:
        marks_2['tat_visible'][row] = 1
    else:
        marks_2['tat_visible'][row] = 0
        
        
for row in range(len(marks_2)):
    if any(substring in marks_2['mark'][row] for substring in substring_list_mc) == True:
        marks_2['mc_visible'][row] = 1
    else:
        marks_2['mc_visible'][row] = 0


for row in range(len(marks_2)):
    if marks_2['tat_visible'][row] == 1 or marks_2['mc_visible'][row] == 1  or marks_2['scar_visible'][row] == 1 :
        marks_2['any_visible'][row] = 1
    else:
        marks_2['any_visible'][row] = 0
        
marks_2

,id,mark,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,scar_visible,tat_visible,mc_visible,any_visible
0,A00147,"SCAR, ARM, RIGHT - 2\""",NaN,NaN,NaN,NaN,1,0,0,1
1,A00147,"SCAR, WRIST, RIGHT - 1\""",NaN,NaN,NaN,NaN,1,0,0,1
2,A00147,"TATTOO, ARM, LEFT - 1111",NaN,NaN,NaN,NaN,0,1,0,1
3,A00220,"SCAR, ARM, LEFT - SCAR ARM, LEFT",NaN,NaN,NaN,NaN,1,0,0,1
4,A00220,"SCAR, CHEST - SCAR CHEST",NaN,NaN,NaN,NaN,0,0,0,0
5,A00360,"SCAR, EYEBROW, LEFT (EYE AREA) - SCAR EYEBROW,...",NaN,NaN,NaN,NaN,0,0,0,0
6,A00360,"TATTOO, SHOULDER, LEFT - ROSE",NaN,NaN,NaN,NaN,0,0,0,0
7,A00367,"SCAR, FOREHEAD - GLASS CUT",NaN,NaN,NaN,NaN,1,0,0,1
8,A01054,"SCAR, ARM, RIGHT - SCAR ARM, RIGHT",NaN,NaN,NaN,NaN,1,0,0,1
9,A01054,"SCAR, HAND, RIGHT - SCAR HAND, RIGHT",NaN,NaN,NaN,NaN,1,0,0,1


### looking at the "Unnamed" columns

In [28]:
marks_2['Unnamed: 2'].unique()

array([nan, 'GUNSHOT WOUND', "'MM'", ..., 'G TRAMMELL\\"',
       ' ARMS, HANDS, FEE', ' ROSE'], dtype=object)

In [29]:
marks_2.loc[marks_2['Unnamed: 2'] == 'GUNSHOT WOUND']

,id,mark,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,scar_visible,tat_visible,mc_visible,any_visible
82,A01906,"TATTOO, CHEEK (FACE), RIGHT - HEART, BACK\",GUNSHOT WOUND,NaN,NaN,NaN,0,1,0,1


In [30]:
marks_2.loc[marks_2['Unnamed: 2'] == ' ARMS, HANDS, FEE']

,id,mark,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,scar_visible,tat_visible,mc_visible,any_visible
187679,Y30274,"PIERCED EARS - 1X EA, N/MARKS\","ARMS, HANDS, FEE",NaN,NaN,NaN,0,0,0,0


In [31]:
marks_2['Unnamed: 3'].unique()

array([nan, ' 100% HUNKY', 'DAGGER', ' $', 'MARG', 'L HND:\\"MOM:',
       ' 1/2 HEART', ' DOIN TIME HOURGLAS', 'FLAG', 'HALOGEN.',
       ' \\"CVL\\",GLASS', 'ESTHER', '7\\', 'RHND:CHANEL', 'DAGGER\\',
       ' TEARDROP', 'CLOUD\\', ' TYSHAY', 'ULA:MY WIF', 'VOID',
       'ABDOM:KATHERINE', ' MOON', ' NANCY', ' XAVIER\\',
       'CROSS,SKULL,ROSE,TIGE', 'RSIDE:WIZARD', 'RHIP:MOSQUITO',
       ' SHENIKA', 'RLEG:187,G', 'LARBESE)', ' SKULL\\', 'PRIDE',
       ' 2 LINES', ' 4PT STAR', 'LFA:NONE', 'FEATHERS)', ' PRAYING HANDS',
       'CHEST:HEART', 'RLEG:LOVE', 'LHND:K', 'TEAR DROP', ' L LEG/DEMON',
       'FLOW), \\"', ' SD, SUICID', ' ALEX, HEART, R', 'PANTHER',
       'LLEG:SHA BAZ', '  L LEG/MAL', 'Scorpion', 'LFA:13', 'BACK:KEENAN',
       'LLEG:SKULLS', ' FLAMES\\', 'AS', 'ULA:HONESTY', 'ULA:CROSS',
       'LFA:MICHELE', 'ULA:P.HAN', 'BRASS KNUCKLES', 'NECK:TAY TAY',
       ' SCAR BACK: SURGICAL', 'LUCKY 13', 'CHEST:TIGE', 'LFA:RIP LA',
       'SKULL\\', 'JOYCE', 'LFA:CLOWN', 'BRUC

In [32]:
marks_2['Unnamed: 4'].unique()

array([nan, 'LLL\\', '+.', ' EYE OF RA', ' WIZARD', ' GUN', 'ALFRIDA',
       ' NICOLE', ' DANE\\', ' KEYASHA', ' BLA', 'COD,', ' ESC, CROWN,',
       ' BITCH,JAMIER', 'GLORI', ' PTG', ' BC', ' YOUNG MONEY', ' DIA\\',
       ' CROWN\\', ' RU', ' SAMORA', ' BPSN\\', 'LLEG:D/TAIL', 'NEY',
       'AARON\\', ' SLEEVED', '$MERK\\', '5PTSTA', ' 8 BALL', 'URA:J',
       'MUSIC', ' DESTANEY', 'KEIAHRA', ' RIP\\', 'LISETT', '\\"GET AT M',
       'OTF\\', 'ANKLES', '$', ' MICHAEL ON SIDE', 'CRYSTAL', 'PRINCES',
       'RIP JA MANUYIA', 'DIAMON', 'RG', 'VINNI', 'TIGER, e', 'FLOWERS'],
      dtype=object)

In [33]:
marks_2['Unnamed: 5'].unique()

array([nan, 'FORKS', ' JESTER', ' CLUB)', ' $', ' IN', 'HEARTS', ' DANA',
       1], dtype=object)

I think this information is not helpful for us, so I drop it

In [34]:
marks_2.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5'], axis = 'columns', inplace = False)
marks_2.head()

,id,mark,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,scar_visible,tat_visible,mc_visible,any_visible
0,A00147,"SCAR, ARM, RIGHT - 2\""",NaN,NaN,NaN,NaN,1,0,0,1
1,A00147,"SCAR, WRIST, RIGHT - 1\""",NaN,NaN,NaN,NaN,1,0,0,1
2,A00147,"TATTOO, ARM, LEFT - 1111",NaN,NaN,NaN,NaN,0,1,0,1
3,A00220,"SCAR, ARM, LEFT - SCAR ARM, LEFT",NaN,NaN,NaN,NaN,1,0,0,1
4,A00220,"SCAR, CHEST - SCAR CHEST",NaN,NaN,NaN,NaN,0,0,0,0


In [35]:
marks_2.head()

,id,mark,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,scar_visible,tat_visible,mc_visible,any_visible
0,A00147,"SCAR, ARM, RIGHT - 2\""",NaN,NaN,NaN,NaN,1,0,0,1
1,A00147,"SCAR, WRIST, RIGHT - 1\""",NaN,NaN,NaN,NaN,1,0,0,1
2,A00147,"TATTOO, ARM, LEFT - 1111",NaN,NaN,NaN,NaN,0,1,0,1
3,A00220,"SCAR, ARM, LEFT - SCAR ARM, LEFT",NaN,NaN,NaN,NaN,1,0,0,1
4,A00220,"SCAR, CHEST - SCAR CHEST",NaN,NaN,NaN,NaN,0,0,0,0


In [36]:
#Save new csv
marks_2.to_csv('marks_2.csv')

### Person

In [37]:
person = pd.read_excel('person.xlsx')
person.head()

,id,name,date_of_birth,weight,hair,sex,height,race,eyes,admission_date,...,last_paroled_date,projected_discharge_date,parole_date,electronic_detention_date,discharge_date,parent_institution,offender_status,location,sex_offender_registry_required,alias
0,A00147,"MCCUTCHEON, JOHN",06/14/1949,185.0,Brown,Male,67.0,White,Blue,02/16/1983,...,NaN,2036-06-10 00:00:00,NaN,NaN,NaN,DIXON CORRECTIONAL CENTER,IN CUSTODY,DIXON,1.0,NaN
1,A00220,"WALKER, ISIAH",03/30/1957,155.0,Black,Male,73.0,Black,Brown,05/19/2016,...,NaN,NaN,NaN,NaN,NaN,STATEVILLE CORRECTIONAL CENTER,NON-IDOC CUSTODY,ILL/OTH STATE/FED CONCURR,NaN,NaN
2,A00360,"BELL, HOWARD",12/18/1946,167.0,Gray or Partially Gray,Male,69.0,White,Green,02/26/1988,...,NaN,TO BE DETERMINED,2017-02-10 00:00:00,NaN,NaN,PINCKNEYVILLE CORRECTIONAL CENTER,PAROLE,PAROLE DISTRICT 1,NaN,HOWARD R BELL | DONALD BROADSTONE | RONALD B...
3,A00367,"GARVIN, RAYMOND",1954-12-01 00:00:00,245.0,Black,Male,72.0,Black,Brown,2017-09-11 00:00:00,...,NaN,11/20/2020,NaN,NaN,NaN,WESTERN ILLINOIS CORRECTIONAL CENTER,IN CUSTODY,WESTERN ILLINOIS,NaN,NaN
4,A01054,"TIPTON, DARNELL",03/25/1954,166.0,Salt and Pepper,Male,67.0,Black,Brown,12/23/1988,...,NaN,08/14/2068,NaN,NaN,NaN,MENARD CORRECTIONAL CENTER,IN CUSTODY,MENARD,1.0,NaN


changing the measures from imperial to reasonable

In [38]:
person["height"] = (person["height"]*2.54)
person["height"].head()

0    170.18
1    185.42
2    175.26
3    182.88
4    170.18
Name: height, dtype: float64

In [39]:
person["weight"] = (person["weight"]*0.45359237)
person["weight"].head()

0     83.914588
1     70.306817
2     75.749926
3    111.130131
4     75.296333
Name: weight, dtype: float64

In [40]:
person = person.drop(['alias', 'electronic_detention_date', 'discharge_date', 'parole_date', 'projected_discharge_date', 'last_paroled_date', 'projected_parole_date'], axis = 'columns', inplace = False)
person.head()

,id,name,date_of_birth,weight,hair,sex,height,race,eyes,admission_date,parent_institution,offender_status,location,sex_offender_registry_required
0,A00147,"MCCUTCHEON, JOHN",06/14/1949,83.914588,Brown,Male,170.18,White,Blue,02/16/1983,DIXON CORRECTIONAL CENTER,IN CUSTODY,DIXON,1.0
1,A00220,"WALKER, ISIAH",03/30/1957,70.306817,Black,Male,185.42,Black,Brown,05/19/2016,STATEVILLE CORRECTIONAL CENTER,NON-IDOC CUSTODY,ILL/OTH STATE/FED CONCURR,NaN
2,A00360,"BELL, HOWARD",12/18/1946,75.749926,Gray or Partially Gray,Male,175.26,White,Green,02/26/1988,PINCKNEYVILLE CORRECTIONAL CENTER,PAROLE,PAROLE DISTRICT 1,NaN
3,A00367,"GARVIN, RAYMOND",1954-12-01 00:00:00,111.130131,Black,Male,182.88,Black,Brown,2017-09-11 00:00:00,WESTERN ILLINOIS CORRECTIONAL CENTER,IN CUSTODY,WESTERN ILLINOIS,NaN
4,A01054,"TIPTON, DARNELL",03/25/1954,75.296333,Salt and Pepper,Male,170.18,Black,Brown,12/23/1988,MENARD CORRECTIONAL CENTER,IN CUSTODY,MENARD,1.0


lets change the dates to uniform format

In [41]:
from dateutil.parser import parse

def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(str(string), fuzzy=fuzzy)
        return True

    except ValueError:
        return False

persons_2 = person

count = 0
list_of_errors = []
for row in range(len(persons_2)):
    if is_date(persons_2['date_of_birth'][row], fuzzy=False) == False:
        count = count+1
        list_of_errors.append(row)
    else:
        count = count
    

for row in range(len(persons_2)):
    if is_date(persons_2['admission_date'][row], fuzzy=False) == False:
        count = count+1
        list_of_errors.append(row)
    else:
        count = count
        
        
print('There are ' + str(count) + ' values, that are not datetime format')

There are 56 values, that are not datetime format


In [42]:
print(list_of_errors)

[1288, 1645, 2251, 2423, 3579, 3722, 4263, 4391, 6374, 6472, 7277, 9000, 9012, 9323, 14237, 14486, 15605, 15607, 20902, 22543, 24124, 24288, 26269, 26363, 27074, 28517, 28518, 28558, 28696, 28721, 29786, 30231, 30364, 30799, 31060, 31445, 32247, 33459, 33835, 38602, 46007, 46727, 48384, 48510, 49997, 50501, 50576, 50788, 50809, 54884, 55527, 55918, 55946, 57324, 58007, 61109]


Yes, this is not dynamic, but I am too lazy, sorry

In [43]:
persons_2 = persons_2.drop([1288, 1645, 2251, 2423, 3579, 3722, 4263, 4391, 6374, 6472, 7277, 9000, 9012, 9323, 14237, 14486, 15605, 15607, 20902, 22543, 24124, 24288, 26269, 26363, 27074, 28517, 28518, 28558, 28696, 28721, 29786, 30231, 30364, 30799, 31060, 31445, 32247, 33459, 33835, 38602, 46007, 46727, 48384, 48510, 49997, 50501, 50576, 50788, 50809, 54884, 55527, 55918, 55946, 57324, 58007, 61109])
print(len(persons_2))
persons_2['date_of_birth'] = pd.to_datetime(persons_2.date_of_birth)
persons_2['admission_date'] = pd.to_datetime(persons_2.admission_date)

61054


In [44]:
persons_2.head(10)

,id,name,date_of_birth,weight,hair,sex,height,race,eyes,admission_date,parent_institution,offender_status,location,sex_offender_registry_required
0,A00147,"MCCUTCHEON, JOHN",1949-06-14,83.914588,Brown,Male,170.18,White,Blue,1983-02-16,DIXON CORRECTIONAL CENTER,IN CUSTODY,DIXON,1.0
1,A00220,"WALKER, ISIAH",1957-03-30,70.306817,Black,Male,185.42,Black,Brown,2016-05-19,STATEVILLE CORRECTIONAL CENTER,NON-IDOC CUSTODY,ILL/OTH STATE/FED CONCURR,NaN
2,A00360,"BELL, HOWARD",1946-12-18,75.749926,Gray or Partially Gray,Male,175.26,White,Green,1988-02-26,PINCKNEYVILLE CORRECTIONAL CENTER,PAROLE,PAROLE DISTRICT 1,NaN
3,A00367,"GARVIN, RAYMOND",1954-12-01,111.130131,Black,Male,182.88,Black,Brown,2017-09-11,WESTERN ILLINOIS CORRECTIONAL CENTER,IN CUSTODY,WESTERN ILLINOIS,NaN
4,A01054,"TIPTON, DARNELL",1954-03-25,75.296333,Salt and Pepper,Male,170.18,Black,Brown,1988-12-23,MENARD CORRECTIONAL CENTER,IN CUSTODY,MENARD,1.0
5,A01072,"BRISBON, HENRY",1956-12-01,88.450512,Black,Male,175.26,Black,Brown,1974-02-22,PONTIAC CORRECTIONAL CENTER,IN CUSTODY,PONTIAC,NaN
6,A01077,"JONES, ROBERT",1953-03-04,81.646627,Black,Male,172.72,Black,Brown,1983-09-16,PINCKNEYVILLE CORRECTIONAL CENTER,IN CUSTODY,PINCKNEYVILLE,NaN
7,A01148,"BROWN, RONALD",1955-02-02,77.110703,Black,Male,180.34,Black,Brown,2005-12-13,MENARD CORRECTIONAL CENTER,IN CUSTODY,MENARD,NaN
8,A01157,"LANGFORD, ROBERT",1945-01-18,93.893621,Black,Male,182.88,Black,Black,2000-09-15,STATEVILLE CORRECTIONAL CENTER,IN CUSTODY,STATEVILLE,NaN
9,A01181,"WILLIAMS, MELVIN",1955-10-09,86.636143,Black,Male,180.34,Black,Brown,2007-10-08,STATEVILLE CORRECTIONAL CENTER,IN CUSTODY,STATEVILLE,NaN


In [45]:
persons_2['Age_admission'] = persons_2['admission_date'] - persons_2['date_of_birth']
persons_2.head(10)

,id,name,date_of_birth,weight,hair,sex,height,race,eyes,admission_date,parent_institution,offender_status,location,sex_offender_registry_required,Age_admission
0,A00147,"MCCUTCHEON, JOHN",1949-06-14,83.914588,Brown,Male,170.18,White,Blue,1983-02-16,DIXON CORRECTIONAL CENTER,IN CUSTODY,DIXON,1.0,12300 days
1,A00220,"WALKER, ISIAH",1957-03-30,70.306817,Black,Male,185.42,Black,Brown,2016-05-19,STATEVILLE CORRECTIONAL CENTER,NON-IDOC CUSTODY,ILL/OTH STATE/FED CONCURR,NaN,21600 days
2,A00360,"BELL, HOWARD",1946-12-18,75.749926,Gray or Partially Gray,Male,175.26,White,Green,1988-02-26,PINCKNEYVILLE CORRECTIONAL CENTER,PAROLE,PAROLE DISTRICT 1,NaN,15045 days
3,A00367,"GARVIN, RAYMOND",1954-12-01,111.130131,Black,Male,182.88,Black,Brown,2017-09-11,WESTERN ILLINOIS CORRECTIONAL CENTER,IN CUSTODY,WESTERN ILLINOIS,NaN,22930 days
4,A01054,"TIPTON, DARNELL",1954-03-25,75.296333,Salt and Pepper,Male,170.18,Black,Brown,1988-12-23,MENARD CORRECTIONAL CENTER,IN CUSTODY,MENARD,1.0,12692 days
5,A01072,"BRISBON, HENRY",1956-12-01,88.450512,Black,Male,175.26,Black,Brown,1974-02-22,PONTIAC CORRECTIONAL CENTER,IN CUSTODY,PONTIAC,NaN,6292 days
6,A01077,"JONES, ROBERT",1953-03-04,81.646627,Black,Male,172.72,Black,Brown,1983-09-16,PINCKNEYVILLE CORRECTIONAL CENTER,IN CUSTODY,PINCKNEYVILLE,NaN,11153 days
7,A01148,"BROWN, RONALD",1955-02-02,77.110703,Black,Male,180.34,Black,Brown,2005-12-13,MENARD CORRECTIONAL CENTER,IN CUSTODY,MENARD,NaN,18577 days
8,A01157,"LANGFORD, ROBERT",1945-01-18,93.893621,Black,Male,182.88,Black,Black,2000-09-15,STATEVILLE CORRECTIONAL CENTER,IN CUSTODY,STATEVILLE,NaN,20329 days
9,A01181,"WILLIAMS, MELVIN",1955-10-09,86.636143,Black,Male,180.34,Black,Brown,2007-10-08,STATEVILLE CORRECTIONAL CENTER,IN CUSTODY,STATEVILLE,NaN,18992 days


from datetime import datetime
def yearsago(years, from_date=None):
    if from_date is None:
        from_date = datetime.now()
    try:
        return from_date.replace(year=from_date.year - years)
    except ValueError:
        # Must be 2/29!
        assert from_date.month == 2 and from_date.day == 29 # can be removed
        return from_date.replace(month=2, day=28,
                                 year=from_date.year-years)

In [46]:
persons_2.to_csv('persons_2.csv')

## Sentencing

lets fix the dates in the same manner as before

In [47]:
sentencing = pd.read_excel('sentencing.xlsx')
sentencing_2 = sentencing
sentencing.head()

,id,mittimus,class,count,offense,custody_date,sentence,county,sentence_discharged
0,A00147,82CF32,1,2,AGGR KIDNAPPING-NO RANSOM,11/26/1982,30 Years 0 Months 0 Days,JASPER,NO
1,A00147,82CF32,X,1,ATTEMPT MURDER/INTENT TO KILL/INJURE,11/26/1982,50 Years 0 Months 0 Days,JASPER,NO
2,A00147,82CF32,X,1,RAPE,11/26/1982,50 Years 0 Months 0 Days,JASPER,NO
3,A00147,82CF32,2,1,BURGLARY,11/26/1982,14 Years 0 Months 0 Days,JASPER,NO
4,A00220,7916505,3,1,ATTEMPT BURGLARY,00/00/0000,2 Years 0 Months 0 Days,COOK,YES


In [48]:
list_of_errors = []
count = 0

for row in range(len(sentencing_2)):
    if is_date(sentencing_2['custody_date'][row], fuzzy=False) == False:
        count = count+1
        list_of_errors.append(row)
    else:
        count = count

print('There are ' + str(count) + ' values, that are not datetime format')

list_of_zeros  = []
count_zeroes = 0

for row in range(len(sentencing_2)):
    if sentencing_2['custody_date'][row] == '00/00/0000':
        count_zeroes = count_zeroes+1
        list_of_zeros.append(row)
    else:
        count_zeroes = count_zeroes
                
print('There are ' + str(count_zeroes) + ' values, that are zero')

There are 59 values, that are not datetime format
There are 59 values, that are zero


In [49]:
print(list_of_zeros)

[4, 5, 263, 264, 330, 765, 766, 1045, 1046, 1782, 2317, 2318, 2319, 2397, 2417, 2418, 2600, 2601, 3153, 3154, 3155, 3271, 3272, 3329, 3330, 3366, 3406, 3573, 4893, 4945, 4946, 5251, 5252, 5323, 5947, 5989, 6021, 6310, 6311, 6312, 6313, 6353, 6354, 6355, 130628, 130822, 131243, 131477, 131478, 131559, 131631, 131770, 131771, 131772, 131954, 131955, 131956, 133169, 133254]


In [50]:
print(list_of_errors)

[4, 5, 263, 264, 330, 765, 766, 1045, 1046, 1782, 2317, 2318, 2319, 2397, 2417, 2418, 2600, 2601, 3153, 3154, 3155, 3271, 3272, 3329, 3330, 3366, 3406, 3573, 4893, 4945, 4946, 5251, 5252, 5323, 5947, 5989, 6021, 6310, 6311, 6312, 6313, 6353, 6354, 6355, 130628, 130822, 131243, 131477, 131478, 131559, 131631, 131770, 131771, 131772, 131954, 131955, 131956, 133169, 133254]


In [51]:
sentencing_2 = sentencing_2.drop([4, 5, 263, 264, 330, 765, 766, 1045, 1046, 1782, 2317, 2318, 2319, 2397, 2417, 2418, 2600, 2601, 3153, 3154, 3155, 3271, 3272, 3329, 3330, 3366, 3406, 3573, 4893, 4945, 4946, 5251, 5252, 5323, 5947, 5989, 6021, 6310, 6311, 6312, 6313, 6353, 6354, 6355, 130628, 130822, 131243, 131477, 131478, 131559, 131631, 131770, 131771, 131772, 131954, 131955, 131956, 133169, 133254])
        
sentencing_2['custody_date'] = pd.to_datetime(sentencing_2.custody_date)
sentencing_2.head()

,id,mittimus,class,count,offense,custody_date,sentence,county,sentence_discharged
0,A00147,82CF32,1,2,AGGR KIDNAPPING-NO RANSOM,1982-11-26,30 Years 0 Months 0 Days,JASPER,NO
1,A00147,82CF32,X,1,ATTEMPT MURDER/INTENT TO KILL/INJURE,1982-11-26,50 Years 0 Months 0 Days,JASPER,NO
2,A00147,82CF32,X,1,RAPE,1982-11-26,50 Years 0 Months 0 Days,JASPER,NO
3,A00147,82CF32,2,1,BURGLARY,1982-11-26,14 Years 0 Months 0 Days,JASPER,NO
6,A00360,87CR0539101,X,1,ARMED ROBBERY,1987-12-04,60 Years 0 Months 0 Days,COOK,NO


In [52]:
sentencing_2.dropna(subset = ["sentence"], inplace=True)

In [53]:
from collections import Counter
Counter(sentencing_2.sentence)

Counter({'30 Years 0 Months 0 Days': 3041,
         '50 Years 0 Months 0 Days': 984,
         '14 Years 0 Months 0 Days': 1672,
         '60 Years 0 Months 0 Days': 1248,
         '1 Years 0 Months 0 Days': 20332,
         '2 Years 0 Months 0 Days': 27728,
         '40 Years 0 Months 0 Days': 929,
         '3 Years 0 Months 0 Days': 40338,
         '0 Years 32 Months 0 Days': 40,
         '0 Years 28 Months 0 Days': 78,
         '6 Years 6 Months 0 Days': 657,
         '7 Years 0 Months 0 Days': 9097,
         '0 Years 16 Months 0 Days': 121,
         '10 Years 0 Months 0 Days': 7875,
         '4 Years 6 Months 0 Days': 1968,
         '5 Years 0 Months 0 Days': 15550,
         '4 Years 0 Months 0 Days': 26632,
         '3 Years 6 Months 0 Days': 2528,
         '9 Years 0 Months 0 Days': 3581,
         '15 Years 0 Months 0 Days': 4376,
         'LIFE': 1294,
         '20 Years 0 Months 0 Days': 3210,
         '3000 Years 0 Months 0 Days': 2,
         '6 Years 0 Months 0 Days': 21994,
  

In [54]:
len(sentencing_2)

244621

In [55]:
sentencing_2.to_csv('sentencing_2.csv')


In [56]:
sentencing_2.head(100)

,id,mittimus,class,count,offense,custody_date,sentence,county,sentence_discharged
0,A00147,82CF32,1,2,AGGR KIDNAPPING-NO RANSOM,1982-11-26,30 Years 0 Months 0 Days,JASPER,NO
1,A00147,82CF32,X,1,ATTEMPT MURDER/INTENT TO KILL/INJURE,1982-11-26,50 Years 0 Months 0 Days,JASPER,NO
2,A00147,82CF32,X,1,RAPE,1982-11-26,50 Years 0 Months 0 Days,JASPER,NO
3,A00147,82CF32,2,1,BURGLARY,1982-11-26,14 Years 0 Months 0 Days,JASPER,NO
6,A00360,87CR0539101,X,1,ARMED ROBBERY,1987-12-04,60 Years 0 Months 0 Days,COOK,NO
7,A00360,01CF187,4,1,OBSTRUCTING JUSTICE,1987-12-04,1 Years 0 Months 0 Days,FULTON,NO
8,A00360,84C10405,3,1,CARRY/POSS FIREARM/PUBLIC/SCH,1984-08-08,2 Years 0 Months 0 Days,COOK,YES
9,A00360,651839,2,1,BURGLARY,1968-04-05,40 Years 0 Months 0 Days,COOK,YES
10,A00360,682118,M,1,MURDER/INTENT TO KILL/INJURE,1968-04-05,40 Years 0 Months 0 Days,COOK,YES
11,A00367,17C55044701,2,1,THEFT CONTROL INTENT 10K<100K,2017-07-29,3 Years 0 Months 0 Days,COOK,NO


In [58]:
#merged = pd.merge(persons_2, marks_2,  on="id")
#merged = pd.merge(merged, sentencing_2,  on="id")

In [ ]:
print('dataframe sentencing is ' + str(len(sentencing)) + ' rows long')
print('dataframe merged is ' + str(len(merged)) + ' rows long')

In [62]:
#merged.to_csv('merged_person_sentencing.csv')
os.listdir()

['.ipynb_checkpoints',
 '436801_829570_bundle_archive.zip',
 'datasets',
 'datasets_251_561_student-mat.csv',
 'Data_cleaning.ipynb',
 'desktop.ini',
 'front',
 'htmltocsv.py',
 'illinois_doc_dataset.torrent',
 'images',
 'inmates',
 'M1',
 'M2',
 'marks.csv',
 'marks.xlsx',
 'marks_2',
 'marks_2.csv',
 'merged',
 'merged_person_sentencing.csv',
 'notebook_M1D4.ipynb',
 'person.csv',
 'person.xlsx',
 'persons_2',
 'persons_2.csv',
 'Python for ML',
 'readme',
 'sentencing.csv',
 'sentencing.xlsx',
 'sentencing_2',
 'sentencing_2.csv',
 'sentencing_2.xlsx',
 'side',
 'Untitled.ipynb',
 'Untitled1.ipynb']